# Parsing top-down direzionale (caso generale)

In [ ]:
from liblet import (
    Production, 
    Grammar, 
    Derivation, 
    ProductionGraph, 
    Queue, 
    Stack, 
    dod2table,
    animate_derivation,
    __version__
)
__version__

'1.2.0-beta'

## Imitando le derivazioni leftmost…

In [ ]:
# fig. 6.1, pag. 165

G = Grammar.from_string("""
S -> a B | b A
A -> a | a S | b A A
B -> b | b S | a B B
""")
G

Grammar(N={A, B, S}, T={a, b}, P=(S -> a B, S -> b A, A -> a, A -> a S, A -> b A A, B -> b, B -> b S, B -> a B B), S=S)

In [ ]:
# parola aabb

animate_derivation(Derivation(G).leftmost((0, 7, 5, 5)))

In [ ]:
# non presente nel libro (caso che mostra meglio l'esigenza di una pila)

G = Grammar.from_string("""
S -> a B C
B -> a B | b
C -> a
""")

In [ ]:
# parola aaba

animate_derivation(Derivation(G).leftmost((0, 1, 2, 3)))

# Simulare la computazione del NPDA 

La simulazione si basa su visite del DAG delle computazioni. In ogni nodo conserviamo la descrizione istantanea (a cui abbiamo aggiunto la derivazione che ha condotto a tale derivazione).

In [ ]:
# mostriamo le derivazioni similmente a quanto fatto
# nel libro di testo

def show_instdescrs(store):
    for deriv, stack, rest in store:
        print([p for p, q in deriv.steps()], ''.join(reversed(list(stack))), rest)
    print('-' * 60)

## La funzione "stato prossimo"

In [ ]:
def make_next_instdescrs(G):
    def next_instdescrs(curr):
        deriv, stack, rest = curr
        top = stack.pop()
        if top == rest[0] == '#': 
            # la parola è terminata e la pila è vuota
            return [(deriv, None, None)]
        if top in G.T:
            # MATCH
            if top == rest[0]: 
                return [(deriv, update_stack(stack, None), rest[1:])]
            else:
                return []
        else:
            # PREDICT
            return [
                (deriv.leftmost(P), update_stack(stack, P), rest)
                for P in filter(Production.such_that(lhs = top), G.P)
            ]
    return next_instdescrs

# tengo da parte un riferimento a questa implementazione per dopo…
original_make_next_instdescrs = make_next_instdescrs

L'aggiornamento della derivazione è elementare, in quanto la derivazione è *immutabile*; aggiornare lo stack richiede più attenzione perché, essendo *mutabile* dobbiamo essere certi che i passi successivi della visita non alterino lo stack corrispondente ad uno dei passi precedenti.


In [ ]:
def update_stack(stack, P):
    copy = stack.copy()
    if P is None: return copy
    for X in reversed(P.rhs): copy.push(X)
    return copy

## Usando una visita in ampiezza

Visitiamo il DAG con una *coda*, teniamo da parte *tutte le derivazioni* che man mano troviamo. 

La visita "verbosa" stampa il contenuto dell'intera coda ad ogni iterazione.

Il parametro `first_only` fa in modo che la simulazione si arresti quando viene trovata la prima derivazione.

In [ ]:
def breadth_first(G, word, verbose = False, first_only = False):
    next_instdescrs = make_next_instdescrs(G)
    q = Queue()
    q.enqueue((Derivation(G), Stack(['#', G.S]), word + '#'))
    derivations = [] # le derivazioni raccolte man mano
    while q:
        if verbose: show_instdescrs(q)
        curr = q.dequeue()
        for nxt in next_instdescrs(curr): 
            deriv, stack, rest = nxt
            if rest is None:
                derivations.append(deriv)
                if first_only: return derivations
            else:
                q.enqueue(nxt)
    return derivations

In [ ]:
# testiamolo sull'ultima grammatica vista

breadth_first(G, 'aaba')

[S -> a B C -> a a B C -> a a b C -> a a b a]

## Non solo GNF

A ben guardare, la simulazione "funziona" non solo per le GNF… se il lato destro mescola simboli terminali e non, finiranno nella pila del PDA e verranno gestiti (apparentemente) senza particolari problemi…

In [ ]:
# un altro test, fedele al libro di testo

# fig. 6.6, pag. 171

G = Grammar.from_string("""
S -> A B | D C 
A -> a | a A
B -> b c | b B c 
D -> a b | a D b 
C -> c | c C
""")
G

Grammar(N={A, B, C, D, S}, T={a, b, c}, P=(S -> A B, S -> D C, A -> a, A -> a A, B -> b c, B -> b B c, D -> a b, D -> a D b, C -> c, C -> c C), S=S)

In [ ]:
breadth_first(G, 'aabc', verbose = True)

[] S# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1] DC# aabc#
------------------------------------------------------------
[1] DC# aabc#
[0, 2] aB# aabc#
[0, 3] aAB# aabc#
------------------------------------------------------------
[0, 2] aB# aabc#
[0, 3] aAB# aabc#
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
------------------------------------------------------------
[0, 3] aAB# aabc#
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
[0, 2] B# abc#
------------------------------------------------------------
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
[0, 2] B# abc#
[0, 3] AB# abc#
------------------------------------------------------------
[1, 7] aDbC# aabc#
[0, 2] B# abc#
[0, 3] AB# abc#
[1, 6] bC# abc#
------------------------------------------------------------
[0, 2] B# abc#
[0, 3] AB# abc#
[1, 6] bC# abc#
[1, 7] DbC# abc#
------------------------------------------------------------
[0, 3] AB# abc#
[1, 6] bC# abc#
[1, 7] DbC# abc#
[0, 2, 4] bc# abc#
[0, 2, 5] bBc# 

[S -> A B -> a A B -> a a B -> a a b c]

## Usando una visita in profondità

Visitiamo il DAG con una *pila*, come prima teniamo da parte *tutte le derivazioni* che man mano troviamo. 

La visita "verbosa" stampa il contenuto dell'intera coda ad ogni iterazione.

Il parametro `max_steps` limita (se diverso da -1) il numero massimo di passi di simulazione.

In [ ]:
def depth_first(G, word, verbose = False, max_steps = -1):
    next_instdescrs = make_next_instdescrs(G)
    s = Stack()
    s.push((Derivation(G), Stack(['#', G.S]), word + '#'))
    derivations = []
    steps = 0
    while s:
        if steps > max_steps > -1: break
        steps += 1
        if verbose: show_instdescrs(s)
        curr = s.pop()
        for nxt in next_instdescrs(curr): 
            deriv, stack, rest = nxt
            if rest is None:
                derivations.append(deriv)
            else:
                s.push(nxt)
    return derivations

In [ ]:
# Ancora l'esempio con la grammatica di fig. 6.6, pag. 171

depth_first(G, 'aabc', verbose = True)

[] S# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1] DC# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7] DbC# abc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7] aDbbC# abc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7] DbbC# bc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7, 6] abbbC# bc#
[1, 7, 7, 7] aDbbbC# bc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7, 6] abbbC# bc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abb

[S -> A B -> a A B -> a a B -> a a b c]

## E la ricorsione?

Che succede se la grammatica contiene produzioni ricorsive a sinistra, ossia della forma $A\to A\gamma$ (con $A$ raggiungibile)? 

Per prima cosa, osserviamo che il DAG delle computazioni diventa *infinito* in quanto deve contenere, per ogni $k>0$, tutte le derivazioni della forma $S\stackrel{*}{\to} \alpha A \beta \stackrel{k}{\to} \alpha A^k \gamma \beta$.

In [ ]:
# pag. 173

G = Grammar.from_string('S -> a | S b')
G

Grammar(N={S}, T={a, b}, P=(S -> a, S -> S b), S=S)

Se ci contentiamo della prima derivazione, sicuramente la visita in ampiezza la troverà

In [ ]:
breadth_first(G, 'ab', first_only = True)

[S -> S b -> a b]

Diverso è il discorso delle visita in profondità: se sulla pila c'è $S$ e la prossima mossa è data da $S -> Sb$ l'altezza della pila cresce senza mai consumare alcun carattere dell'input. 

Per questa ragione, la simulazione non potrà scoprire la derivazione in nessun numero finito di passi!

In [ ]:
depth_first(G, 'ab', verbose = False, max_steps = 100) # non cambia mettendo numeri maggiori!

[]

### <span style="color: red">Rfilessione per casa</span>

Cambia qualcosa in presenza di altre forme di produzioni ricorsive, ossia della forma $A -> \alpha A \beta$? È importante sapere se $\alpha \stackrel{*}{\to} \varepsilon$ o meno? 

Per rispondere, pensate a cosa accade con le GNF (con cui la simulazione funziona sempre), in particolare, ci possono essere produzioni ricorsive nelle GNF? E ricorsive a sinistra?

## Un "trucco"

Se la grammatica non contiene ε-produzioni, si può osservare che se in una configurazione istantanea il numero di simboli sulla pila eccede il numero di simboli della parola che restano da elaborare, quella configurazione non potrà mai condurre ad una derivazione della parola (perché non ci sono abbastanza terminali da associare ai non terminali in pila).

Possiamo modificare la funzione `make_next_instdescrs` in modo tale da "potare" gli stati prossimi in cui la pila è più grande del resto dell'input.


In [ ]:
def productive_instdescrs(descr):
    deriv, stack, rest = descr
    return (stack is None and rest is None) or len(stack) <= len(rest)

def make_next_instdescrs(G):
    next_instdescrs = original_make_next_instdescrs(G)
    def pruned_next_instdescrs(curr):
        return list(filter(productive_instdescrs, next_instdescrs(curr)))
    return pruned_next_instdescrs

Con quetsa modifica funzionano entrambe le visite (senza ulteriori limiti sul numero di derivazioni trovate, o passi effettuati)!

In [ ]:
breadth_first(G, 'ab')

[S -> S b -> a b]

In [ ]:
depth_first(G, 'ab')

[S -> S b -> a b]

### <span style="color: red">Riflessione per casa</span>

Perché questa soluzione è meglio di imporre limiti come `first_only` e `max_steps`?

## Persino l'ambiguità

Nel caso di una grammatica ambiguta (e con produzioni ricorsive), la simulazione con la "potatura" produce tutte le derivazioni…

In [ ]:
# la solita grammatica delle operazioni aritmetiche (ambigua rispetto alla precedenza)

G = Grammar.from_string('E -> E * E | E + E | e')
G

Grammar(N={E}, T={*, +, e}, P=(E -> E * E, E -> E + E, E -> e), S=E)

In [ ]:
breadth_first(G, 'e*e+e')

[E -> E * E -> e * E -> e * E + E -> e * e + E -> e * e + e,
 E -> E + E -> E * E + E -> e * E + E -> e * e + E -> e * e + e]

In [ ]:
depth_first(G, 'e*e+e')

[E -> E + E -> E * E + E -> e * E + E -> e * e + E -> e * e + e,
 E -> E * E -> e * E -> e * E + E -> e * e + E -> e * e + e]

## <span style="color: red;">Riflessioni per casa</span>

Cosa succede se si consentono anche ε-produzioni? Il DAG delle computazioni è finito? Che modi ci possono essere di "potarlo"?